In [1]:
import numpy as np
import tensorflow as tf
import mdn

In [2]:
import tensorflow.keras.backend as K

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Attention
from tensorflow.keras.layers import (Conv2D, Input, Reshape, 
                                     Lambda, Dense, Conv2DTranspose)

In [3]:
import matplotlib.pyplot as plt
import os
import utils
from utils import sampling

In [4]:
import tensorflow_probability as tfp
tfd = tfp.distributions

In [5]:
def Encoder(LATENT_SIZE=32):
    inputs = Input(shape=(64, 64, 3), name='encoder_input')
    h = Conv2D(32, 4, strides=2, activation="relu", name="enc_conv1")(inputs)
    h = Conv2D(64, 4, strides=2, activation="relu", name="enc_conv2")(h)
    h = Conv2D(128, 4, strides=2, activation="relu", name="enc_conv3")(h)
    h = Conv2D(256, 4, strides=2, activation="relu", name="enc_conv4")(h)
    h = Reshape([2*2*256])(h)
    z_mean = Dense(LATENT_SIZE, name='z_mean')(h)
    z_log_var = Dense(LATENT_SIZE, name='z_log_var')(h)
    z = Lambda(sampling, output_shape=(LATENT_SIZE,), name='z')([z_mean, z_log_var])
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    return encoder

In [6]:
seq_len = 128
act_len = 3
n_mixtures = 5
output_dims = 1

In [7]:
trace_dir = './data/traces'
traces = os.listdir(trace_dir)
trace = np.load(os.path.join(trace_dir, traces[0]))
ims = trace['b']
ims = utils.preprocess_images(ims)

In [8]:
e = Encoder()
e.load_weights('./data/weights/encoder_weights.h5')
z = utils.to_latent(e, ims)
actions = trace['a']
pair = np.concatenate((z, actions), axis=1)

In [9]:
latent_dataset = tf.data.Dataset.from_tensor_slices(pair)

In [10]:
sequences = latent_dataset.batch(seq_len + 1, drop_remainder=True)

In [11]:
sequences

<BatchDataset shapes: (129, 35), types: tf.float64>

In [12]:
def split_input_target(chunk):
    input_z = chunk[:-1]
    target_z = chunk[1:, :32]
    return input_z, target_z

In [13]:
dataset = sequences.map(split_input_target)

In [14]:
dataset

<MapDataset shapes: ((128, 35), (128, 32)), types: (tf.float64, tf.float64)>

In [15]:
dataset = dataset.shuffle(10000).batch(utils.BATCH_SIZE, drop_remainder=True)

In [16]:
dataset

<BatchDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

In [17]:
i = 0
for a, b in dataset:
    i += 1

In [18]:
i

0

In [19]:
percent_20 = i // 5
val = dataset.take(percent_20)
train = dataset.skip(percent_20)


In [20]:
val

<TakeDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

In [21]:
train

<SkipDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

## MDN-RNN

In [ ]:
class MDN_RNN(tf.keras.Model):
    def __init__(self, 
                 latent_size=32, 
                 seq_length=128,
                 act_len=3,
                 output_dims=1,
                 n_mixtures=5,
                 hlayer=LSTM,
                 hidden_size=256, 
                 optimizer='adam',):
        
        """Initializer for MDN-RNN"""
        
        super(MDN_RNN, self).__init__()
        self.latent_dim  = latent_size
        self.seq_len     = seq_length
        self.act_len     = act_len
        self.output_dims = output_dims
        self.n_mixtures  = n_mixtures
        self.hlayer      = hlayer
        self.h_size      = hidden_size
        self.optim       = optimizer
        self.T           = 1.25
    
    
    # ----------------------------------------#
    #            LAYER DEFINITIONS            #
    # ----------------------------------------#
    
        self.input_ = InputLayer(
                        batch_size = None,
                        input_shape = (self.seq_len, 
                                       self.act_len + utils.LATENT_SIZE),
                        name='m_input'
                    )
        
        self.rnn =   self.hlayer(self.h_size, return_state=True, 
                                             name='encoder')
        
        self.z_pi    =  Dense(self.n_mixtures * self.output_dims)
        self.z_mu    =  Dense(self.n_mixtures * self.output_dims)
        self.z_sigma =  Dense(self.n_mixtures * self.output_dims)
        
        
        
    def call(self, inputs):
        x = self.input_(inputs)
        r_output, state_h, state_c = self.rnn(x)
        
        pi = self.z_pi(r_output)
        print(pi.shape)
        pi = tf.reshape(pi, shape=(-1, 
                                   self.seq_len, 
                                   self.n_mixtures, 
                                   self.latent_dim))
        pi = tf.math.softmax(pi, axis=2)
        pi = pi / self.T
        
        
        sigma = tf.math.exp(self.z_sigma(r_output))
        sigma = tf.reshape(sigma, shape=(-1, 
                                           self.seq_len, 
                                           self.n_mixtures, 
                                           self.latent_dim))
        sigma = sigma * self.T**0.5
        
        mu = self.z_mu(r_output)
        mu = tf.reshape(mu, shape=(-1, 
                                   self.seq_len, 
                                   self.n_mixtures, 
                                   self.latent_dim))
        # reshape to (-1, seqlen, n_mixtures, latent)

        return pi, sigma, mu           

    def get_config(self):
        config = super(tf.keras.Model, self).get_config()
        config.update({'latent_size': self.latent_dim, 
                       'seq_len':     self.seq_len,
                       'act_len':     self.act_len,
                       'output_dims': self.output_dims,
                       'n_mixtures':  self.n_mixtures,
                       'hlayer':      self.hlayer,
                       'h_size':      self.h_size,
                       'optim':       self.optim})
        return config
        
        

def mdn_loss_func(y_true, y_pred):
        # Reshape inputs in case this is used in a TimeDistribued layer
        y_pred = tf.reshape(y_pred, [-1, (2 * n_mixtures * output_dims) + n_mixtures], name='reshape_ypreds')
        y_true = tf.reshape(y_true, [-1, output_dims], name='reshape_ytrue')
        # Split the inputs into paramaters
        out_mu, out_sigma, out_pi = tf.split(y_pred, num_or_size_splits=[n_mixtures * output_dims,
                                                                         n_mixtures * output_dims,
                                                                         n_mixtures],
                                             axis=-1, name='mdn_coef_split')
        # Construct the mixture models
        cat = tfd.Categorical(logits=out_pi)
        component_splits = [output_dims] * n_mixtures
        mus = tf.split(out_mu, num_or_size_splits=component_splits, axis=1)
        sigs = tf.split(out_sigma, num_or_size_splits=component_splits, axis=1)
        coll = [tfd.MultivariateNormalDiag(loc=loc, scale_diag=scale) for loc, scale
                in zip(mus, sigs)]
        mixture = tfd.Mixture(cat=cat, components=coll)
        loss = mixture.log_prob(y_true)
        loss = tf.negative(loss)
        loss = tf.reduce_mean(loss)
        return loss

M = MDN_RNN() # DOES NOT WORK. 
# M.compile(optimizer=M.optim, loss=mdn_loss_func)

In [ ]:
seq_len = 128
act_len = 3

In [ ]:
OUTPUT_DIMS = 1
N_MIXES     = 5

In [ ]:
M = Sequential([
    Input((seq_len, act_len + utils.LATENT_SIZE)),
    LSTM(256),
    mdn.MDN(OUTPUT_DIMS, N_MIXES)
])

M.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMS, N_MIXES), 
          optimizer=tf.keras.optimizers.Adam()
         )
M.summary()

## Sausage2Seq

In [ ]:
trace_dir = './data/traces'

In [ ]:
traces = os.listdir(trace_dir)

In [ ]:
trace = np.load(os.path.join(trace_dir, traces[0]))

In [ ]:
# BATCH_SIZE = 128
# LATENT_SIZE = 32

In [ ]:
ims = trace['b']

In [ ]:
ims = utils.preprocess_images(ims)

In [ ]:
im = ims[0]

In [ ]:
plt.imshow(im)

In [ ]:
def sausage_to_seq(sausage, seq_len):
    return np.array(np.split(sausage, sausage.shape[0] / seq_len))

In [ ]:
seq_len = 128

In [ ]:
y_true = sausage_to_seq(z, seq_len)

In [ ]:
y_true.shape

In [ ]:
y_true_ = np.array(np.split(y_true, z.shape[0] / utils.BATCH_SIZE))

In [ ]:
y_true = np.array([y_true, y_true])

In [ ]:
y_true_.shape

In [ ]:
actions.shape

In [ ]:
z.shape

In [ ]:
pair = sausage_to_seq(pair, seq_len)

In [ ]:
pair.shape

In [ ]:
y_true.shape

In [ ]:
pair = np.array([pair, pair])

In [ ]:
pair.shape

In [ ]:
pair[0]

In [ ]:
try: 
    M.fit(pair, y_true, batch_size=utils.BATCH_SIZE, epochs=5)
except ValueError as e:
    print(e)